In [1]:
import numpy as np
import pandas as pd
import re
import tensorflow as tf
#Keras
from keras.models import Model, Sequential
from keras import utils
from keras.preprocessing import image
#Keras NLP
from keras.preprocessing.text import Tokenizer, text_to_word_sequence 
from keras.preprocessing.sequence import pad_sequences

#Sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

#Сторонние либы
import time
import os
from PIL import Image

#Базовые слои
from keras.layers import Input, concatenate, Activation, Reshape, Flatten, Dense, LSTM, GRU

#Свертка
from keras.layers import Conv1D, Conv2D, Conv3D, MaxPooling1D, MaxPooling2D, MaxPooling3D
from keras.layers import Conv1DTranspose, Conv2DTranspose, Conv3DTranspose
from keras.layers import GlobalMaxPooling1D, GlobalMaxPooling2D, GlobalMaxPooling3D

#Регуляризация
from keras.layers import Dropout, BatchNormalization
from keras.layers import SpatialDropout1D, SpatialDropout2D, SpatialDropout3D


In [2]:
df = pd.read_csv('hh_fixed.csv')

In [3]:
df.head()

,Unnamed: 0,"Пол, возраст",ЗП,Ищет работу на должность:,Город,Занятость,График,Опыт (двойное нажатие для полной версии),Последенее/нынешнее место работы,Последеняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Авто
0,0,"Мужчина , 29 лет , родился 16 мая 1989",40000 руб.,Специалист по поддержке чата(support team) дом...,"Новороссийск , готов к переезду (Анапа, Геленд...",полная занятость,полный день,Опыт работы 3 года 9 месяцев Специалист по по...,"ООО ""Гольфстрим""",Генеральный директор,Высшее образование 2011 Международный юридиче...,26.04.2019 08:04,Не указано
1,1,"Мужчина , 38 лет , родился 25 мая 1980",40000 руб.,Системный администратор,"Новосибирск , м. Березовая роща , не готов к ...",полная занятость,полный день,Опыт работы 11 лет 11 месяцев Системный админ...,ООО «Завод модульных технологий»,Системный администратор,Высшее образование 2002 Новосибирский государс...,26.04.2019 04:30,Не указано
2,2,"Мужчина , 35 лет , родился 14 июня 1983",300000 руб.,DevOps TeamLead / DevOps архитектор,"Москва , готов к переезду , готов к редким ком...",полная занятость,полный день,Опыт работы 12 лет 11 месяцев DevOps TeamLead...,Банк ВТБ (ПАО),Начальник отдела методологии разработки (DevOp...,DevOps TeamLead / DevOps архитектор 300 000 ру...,09.04.2019 14:40,Не указано
3,3,"Мужчина , 33 года , родился 2 августа 1985",180000 руб.,Руководитель IT отдела,"Москва , м. Щукинская , не готов к переезду ,...","частичная занятость, полная занятость","удаленная работа, полный день",Опыт работы 15 лет 10 месяцев Руководитель IT...,"""Ай-Теко"", ведущий российский системный интегр...",Старший системный администратор,Руководитель IT отдела 180 000 руб. Информацио...,09.04.2019 14:39,Имеется собственный автомобиль
4,4,"Мужчина , 22 года , родился 1 сентября 1996",40000 руб.,Junior Developer,"Москва , м. Юго-Западная , не готов к переезд...","стажировка, частичная занятость, проектная работа","гибкий график, удаленная работа",Опыт работы 1 год 1 месяц Junior Developer 40...,R-Style SoftLab,Менеджер IT-проектов,Junior Developer 40 000 руб. Информационные те...,29.03.2019 12:40,Не указано


In [4]:
df = df.drop(df.columns[0], axis=1)
df.head()

,"Пол, возраст",ЗП,Ищет работу на должность:,Город,Занятость,График,Опыт (двойное нажатие для полной версии),Последенее/нынешнее место работы,Последеняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Авто
0,"Мужчина , 29 лет , родился 16 мая 1989",40000 руб.,Специалист по поддержке чата(support team) дом...,"Новороссийск , готов к переезду (Анапа, Геленд...",полная занятость,полный день,Опыт работы 3 года 9 месяцев Специалист по по...,"ООО ""Гольфстрим""",Генеральный директор,Высшее образование 2011 Международный юридиче...,26.04.2019 08:04,Не указано
1,"Мужчина , 38 лет , родился 25 мая 1980",40000 руб.,Системный администратор,"Новосибирск , м. Березовая роща , не готов к ...",полная занятость,полный день,Опыт работы 11 лет 11 месяцев Системный админ...,ООО «Завод модульных технологий»,Системный администратор,Высшее образование 2002 Новосибирский государс...,26.04.2019 04:30,Не указано
2,"Мужчина , 35 лет , родился 14 июня 1983",300000 руб.,DevOps TeamLead / DevOps архитектор,"Москва , готов к переезду , готов к редким ком...",полная занятость,полный день,Опыт работы 12 лет 11 месяцев DevOps TeamLead...,Банк ВТБ (ПАО),Начальник отдела методологии разработки (DevOp...,DevOps TeamLead / DevOps архитектор 300 000 ру...,09.04.2019 14:40,Не указано
3,"Мужчина , 33 года , родился 2 августа 1985",180000 руб.,Руководитель IT отдела,"Москва , м. Щукинская , не готов к переезду ,...","частичная занятость, полная занятость","удаленная работа, полный день",Опыт работы 15 лет 10 месяцев Руководитель IT...,"""Ай-Теко"", ведущий российский системный интегр...",Старший системный администратор,Руководитель IT отдела 180 000 руб. Информацио...,09.04.2019 14:39,Имеется собственный автомобиль
4,"Мужчина , 22 года , родился 1 сентября 1996",40000 руб.,Junior Developer,"Москва , м. Юго-Западная , не готов к переезд...","стажировка, частичная занятость, проектная работа","гибкий график, удаленная работа",Опыт работы 1 год 1 месяц Junior Developer 40...,R-Style SoftLab,Менеджер IT-проектов,Junior Developer 40 000 руб. Информационные те...,29.03.2019 12:40,Не указано


In [5]:
i=0
for j in range(12):  # Для каждого из столбцов таблицы
    for i in range(df.shape[0]): # По всем резюме
        if type(df.values[i][j])!=float: # если данные текстовые, то заменяем фрагменты кодировки ASCII на отсутствие символов
            df.values[i][j] = df.values[i][j].replace("\xa0","")
            df.values[i][j] = df.values[i][j].replace("\n"," ")

In [17]:
def getParameterSexAge(arg):
    out = [0, 0]
    if 'М' in arg:
        out[0] = 1
    year_tec = 2022
    if len(arg) > 7:
        out[1] = year_tec - int(re.findall(r'\d{4}', arg)[0])
    return out

def getParameterAgeVect(arg):
    outClass =int((arg-13)/5)
    outClass = max(0, min(10,outClass))
    return list(tf.keras.utils.to_categorical(outClass, 11).astype('int'))

def getParameterSalary(arg):
   num = arg
   if (type(num == str)):
      num = re.sub(' ', '', num)
      num = re.sub('[а-яА-ЯёЁ]', '', num)
      num = re.sub('[a-zA-Z]', '', num)
      num = num.replace('.','')

      curr = re.sub('[0-9]','',arg)
      curr = curr.replace('.','').strip()

      # Конвертируем в рубли, если валюта
      if curr == 'USD':
         num = float(num)*65
      elif curr == 'KZT':
         num = float(num)*0.17
      elif curr=='грн':
         num = float(num)*2.6
      elif curr=='белруб':
         num = float(num)*30.5
      elif curr=='EUR':
         num = float(num)*70
      elif curr=='KGS':
         num = float(num)*0.9
      elif curr=='сум':
         num = float(num)*0.007
      elif curr=='AZN':
         num = float(num)*37.5

      salary_int = int(num)
   return salary_int
      
def getParameterCity(arg):
   millionCities = "Новосибирск Екатеринбург Нижний Новгород Казань Челябинск Омск Самара Ростов-на-Дону Уфа Красноярск Пермь Воронеж Волгоград"
   sarg = arg.split(',')         
   for item in sarg:
      if item == 'Москва': return [1, 0, 0 , 0]
      if item == 'Санкт-Петербург': return [0, 1, 0 , 0]
      if item in millionCities: return [0, 0, 1, 0]
   
   return [0, 0, 0, 1]

def getParameterEmployment(arg):
   out = [0, 0, 0, 0]
   if ("стажировка" in arg):           out[0]=1
   if ("частичная занятость" in arg):  out[1]=1
   if ("проектная работа" in arg):     out[2]=1
   if ("полная занятость" in arg):     out[3]=1

   return out

def getParameterSchedule(arg):
  out = [0, 0, 0, 0]
  if ("гибкий график" in arg):    out[0]=1
  if ("полный день" in arg):      out[1]=1
  if ("сменный график" in arg):   out[2]=1
  if ("удаленная работа" in arg): out[3]=1
  return out

def getParameterEducation(arg):
  out = [0, 0, 0, 0]
  if arg in "Высшее Higher education":  out[0] = 1
  if arg in "Среднее специальное":      out[1] = 1
  if arg in "Неоконченное высшее":      out[2] = 1
  if arg in "Среднее образование":      out[3] = 1
  return out

def getParameterExperience(arg):
  arg = str(arg)
  #Проверяем, если не пустая строка
  symbols = 0
  years = 0
  months = 0
  for s in arg:
    if (s != " "):
      symbols += 1
  
  #Находим индексы пробелов около фразы "опыт работы"
  if (symbols > 10):
      spacesIndexes = []
      index = 0
      while ((len(spacesIndexes) < 5) & (index < len(arg))):
          if (arg[index] == " "):
              spacesIndexes.append(index)
          index += 1

      years = 0
      months = 0
      if (arg[spacesIndexes[2]+1] != "м"):
         if (len(spacesIndexes) >= 3):
            yearsStr = arg[spacesIndexes[1]:spacesIndexes[2]] # Записываем в строку значение лет
            years = int(yearsStr)
      
         if (len(spacesIndexes) >= 5):
            monthsStr = arg[spacesIndexes[3]:spacesIndexes[4]] # Записываем в строку значение месяцев
            if(arg[spacesIndexes[2]+1] == "м"):
                months = int(monthsStr)
      else:
        if (len(spacesIndexes) >= 3):
          monthsStr = arg[spacesIndexes[1]:spacesIndexes[2]]
          months = int(monthsStr)
      
  return 12*years+months

def getParameterExperienceVector(arg):
  out = getParameterExperience(arg)
  outClass = 0
  if (out > 6): # если больше 6 месяцев
    outClass = 1
  if (out > 12): # если больше 12 месяцев
    outClass = 2
  if (out > 24): # если больше 24 месяцев
    outClass = 3
  if (out > 36): # если больше 36 месяцев
    outClass = 4
  if (out > 60): # если больше 60 месяцев
    outClass = 5
  if (out > 96): # если больше 96 месяцев
    outClass = 6
  if (out > 120): # если больше 120 месяцев
    outClass = 7
  if (out > 156): # если больше 156 месяцев
    outClass = 8
  if (out > 192): # если больше 192 месяцев
    outClass = 9 
  if (out > 240): # если больше 240 месяцев
    outClass = 10

def getAllParameter(val):
   result = getParameterSexAge(val[0])
   sex = result[0]
   age = getParameterAgeVect(result[1])
   city = getParameterCity(val[3])
   employment = getParameterEmployment(val[4])
   shedule = getParameterSchedule(val[5])
   education = getParameterEducation(val[9])
   out = [] 

   out.append(sex)
   out += age
   out += city
   out += employment
   out += shedule
   out += education
  
   return out

def x_y_Train(values):
   x_train = []
   y_train = []

   for val in values:
      y = getParameterSalary(val[1])

      if y != -1:
         x = getAllParameter(val)
         x_train.append(x)
         y_train.append(y/1000)
   
   xTrain = np.array(x_train)
   yTrain = np.array(y_train)

   return (xTrain, yTrain)

In [18]:
(xTrain, yTrain) = x_y_Train(df.values)

In [8]:
def getJob(arg):
   out = []
   for val in arg:
      text = ''
      text += str(val[2])
      text += ' ' + str(val[6])
      text += ' ' + str(val[9])
   
      if (getParameterSalary(val[1]) != -1):
         out.append(text)
   
   out = np.array(out)
   return out

In [9]:
x_train_text = getJob(df.values)

In [10]:
num_words = 2000
tokenizer = Tokenizer(num_words, oov_token='unknown')
tokenizer.fit_on_texts(x_train_text)
x_train_bow = tokenizer.texts_to_matrix(x_train_text)

In [11]:
yScaler = StandardScaler()
yScaler.fit(yTrain.reshape(-1, 1))
yTrainScaled = yScaler.transform(yTrain.reshape(-1, 1))

In [43]:
def bow():
    inp = Input(shape=(2000))
    x1 = Dense(1000, activation='relu')(inp)
    x1 = Dropout(0.3)(x1)
    x1 = Dense(1200, activation='relu')(x1)
    x1 = Dropout(0.3)(x1)
    x1 = Dense(1000, activation='relu')(x1)
    x1 = Dropout(0.3)(x1)
    x1 = Dense(500, activation='relu')(x1)
    x1 = Dropout(0.3)(x1)
    out = Dense(200, activation='relu')(x1)

    model = Model(inp, out)
    return model

def table():
    inp = Input(shape=(28,))
    x1 = Dense(128, activation='relu')(inp)
    x1 = Dropout(0.3)(x1)
    x1 = Dense(256, activation='relu')(x1)
    x1 = Dropout(0.3)(x1)
    x1 = Dense(100, activation='relu')(x1)
    x1 = Dropout(0.3)(x1)
    out = Dense(128, activation='relu')(x1)

    model = Model(inp, out)
    return model

In [44]:
nn_bow = bow()
nn_table = table()
combined = concatenate([nn_bow.output, nn_table.output])
x = Dense(256, activation='relu')(combined)
x = Dropout(0.3)(x)
x = Dense(528, activation='relu')(x)
x = Dropout(0.3)(x)
out = Dense(1, activation='linear')(x)

model = Model(inputs=[nn_bow.input, nn_table.input], outputs=out)
model.compile(loss='mse', optimizer='rmsprop', metrics=['mae'])
tf.keras.utils.plot_model(model)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [45]:
model.fit([x_train_bow, xTrain], yTrainScaled, batch_size=128, epochs=50, validation_split=0.2)

Epoch 1/50
394/394 [==============================] - 55s 136ms/step - loss: 1.1008 - mae: 0.2700 - val_loss: 0.1095 - val_mae: 0.2303
Epoch 2/50
394/394 [==============================] - 54s 138ms/step - loss: 1.0196 - mae: 0.2129 - val_loss: 0.0993 - val_mae: 0.2109
Epoch 3/50
394/394 [==============================] - 61s 154ms/step - loss: 1.0031 - mae: 0.1928 - val_loss: 0.0994 - val_mae: 0.2095
Epoch 4/50
394/394 [==============================] - 58s 146ms/step - loss: 0.9892 - mae: 0.1762 - val_loss: 0.0832 - val_mae: 0.1807
Epoch 5/50
394/394 [==============================] - 58s 148ms/step - loss: 0.9681 - mae: 0.1605 - val_loss: 0.0845 - val_mae: 0.1934
Epoch 6/50
394/394 [==============================] - 58s 148ms/step - loss: 0.9601 - mae: 0.1481 - val_loss: 0.0748 - val_mae: 0.1749
Epoch 7/50
394/394 [==============================] - 59s 149ms/step - loss: 0.9520 - mae: 0.1410 - val_loss: 0.0794 - val_mae: 0.1795
Epoch 8/50
394/394 [==============================] - 5

KeyboardInterrupt: 